Realizado por: Armando Torner Marchesi

# Logs NASA

In [3]:
sc.stop()

Cargamos librerías:

In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext()

Cargamos los datos

In [6]:
data_file = "./apache.access.log_small"
raw_data = sc.textFile(data_file)

Vemos que hemos cargado bien los datos visualizando los 5 primeros registros

In [7]:
raw_data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

A continuación procedemos con el parseado de los datos:

In [8]:
import re
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1
n_logs = raw_data.count()

In [9]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [10]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])
parsed_def = parsed_rdd.map(lambda line: map_log(line))

### Transformamos en un sqldataframe

In [18]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [32]:
from pyspark.sql import Row
from datetime import datetime

In [40]:
sql_data = parsed_def.map(lambda p: (Row(Host = p[0],
                            Date = datetime.strptime(p[3][:11], "%d/%b/%Y"),
                            Code = p[-2], Endpoint = p[6],
                            Size = p[-1])))                       

In [41]:
lognasa_df = sqlContext.createDataFrame(sql_data)
lognasa_df.registerTempTable("nasa")

In [87]:
df = sqlContext.sql(""" SELECT * FROM nasa LIMIT 15""")
df.show()

+----+-------------------+--------------------+--------------------+-----+
|Code|               Date|            Endpoint|                Host| Size|
+----+-------------------+--------------------+--------------------+-----+
| 200|1995-08-01 00:00:00|/shuttle/missions...|   in24.inetnebr.com| 1839|
| 304|1995-08-01 00:00:00|                   /|     uplherc.upl.com|    0|
| 304|1995-08-01 00:00:00|/images/ksclogo-m...|     uplherc.upl.com|    0|
| 304|1995-08-01 00:00:00|/images/MOSAIC-lo...|     uplherc.upl.com|    0|
| 304|1995-08-01 00:00:00|/images/USA-logos...|     uplherc.upl.com|    0|
| 200|1995-08-01 00:00:00|/images/launch-lo...|ix-esc-ca2-07.ix....| 1713|
| 304|1995-08-01 00:00:00|/images/WORLD-log...|     uplherc.upl.com|    0|
| 200|1995-08-01 00:00:00|/history/skylab/s...|slppp6.intermind.net| 1687|
| 200|1995-08-01 00:00:00|/images/launchmed...|piweba4y.prodigy.com|11853|
| 200|1995-08-01 00:00:00|/history/skylab/s...|slppp6.intermind.net| 9202|
| 200|1995-08-01 00:00:00

In [37]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats()

(count: 3432, mean: 16051.863636363634, stdev: 53247.81574818679, max: 887988.0, min: 0.0)

### Mínimo, media y máximo de las peticiones

In [39]:
medidas = sqlContext.sql(""" 
    SELECT MIN(Size) AS Minimo, MAX(Size) AS Maximo, ROUND(AVG(Size),2) AS Media 
    FROM nasa 
    """)
medidas.show()

+------+------+-------+
|Minimo|Maximo|  Media|
+------+------+-------+
|     -| 99942|16183.9|
+------+------+-------+



### Numero de peticiones de cada código de respuesta

In [93]:
medidas = sqlContext.sql(""" 
    SELECT MIN(Size) AS Minimo, MAX(Size) AS Maximo, ROUND(AVG(Size),2) AS Media 
    FROM nasa 
    """)
medidas.show()

+------+------+-------+
|Minimo|Maximo|  Media|
+------+------+-------+
|     -| 99942|16183.9|
+------+------+-------+



### 20 Host visitados más de 10 veces

In [45]:
hosts_visitaos_mas_diez = sqlContext.sql(""" 
    SELECT Host 
    FROM nasa
    WHERE Size > 10
    ORDER BY Size
    LIMIT 20
    """)
hosts_visitaos_mas_diez.show()

+--------------------+
|                Host|
+--------------------+
|  port13.wavenet.com|
|www-d1.proxy.aol.com|
|       193.84.66.147|
|       ras38.srv.net|
|delorme.richard.i...|
|dd08-050.compuser...|
|        133.43.96.45|
|marimo.kushiro-ct...|
|  rs31-annex3.sfu.ca|
|port26.ts1.msstat...|
|hsccs_gatorbox07....|
|  server.indo.net.id|
|mallard2.duc.aubu...|
|dialup-2-139.gw.u...|
|        hella.stm.it|
|prakinf2.prakinf....|
|ppp1016.po.iijnet...|
|dd08-043.compuser...|
|ad10-015.compuser...|
|ix-min1-02.ix.net...|
+--------------------+



### 10 endpoint más visitados

In [95]:
endpoints_mas_visitados = sqlContext.sql("""
SELECT COUNT(Endpoint), Endpoint
FROM nasa
GROUP BY Endpoint
ORDER BY COUNT (Endpoint) DESC
LIMIT 10""")

endpoints_mas_visitados.show()

+---------------+--------------------+
|count(Endpoint)|            Endpoint|
+---------------+--------------------+
|            167|/images/KSC-logos...|
|            160|/images/NASA-logo...|
|            122|/images/MOSAIC-lo...|
|            120|/images/WORLD-log...|
|            118|/images/USA-logos...|
|            106|/images/ksclogo-m...|
|             85|                   /|
|             74|/history/apollo/i...|
|             69|/images/launch-lo...|
|             66|/images/ksclogosm...|
+---------------+--------------------+



### 10 endpoints más visitados con código distinto de 200

In [98]:
endpoints_mas_visitados_dist_doscien = sqlContext.sql("""
SELECT COUNT(Endpoint), Endpoint
FROM nasa
WHERE Code != 200
GROUP BY Endpoint
ORDER BY COUNT (Endpoint) DESC
LIMIT 10""")
endpoints_mas_visitados_dist_doscien.show()

+---------------+--------------------+
|count(Endpoint)|            Endpoint|
+---------------+--------------------+
|             25|/images/NASA-logo...|
|             24|/images/KSC-logos...|
|             17|/images/MOSAIC-lo...|
|             17|/images/WORLD-log...|
|             16|/images/USA-logos...|
|             10|/images/ksclogo-m...|
|              8|/images/construct...|
|              8|/software/winvn/b...|
|              8|/software/winvn/w...|
|              6|/software/winvn/w...|
+---------------+--------------------+



### Número de hosts distintos

In [52]:
host_distintos = sqlContext.sql(""" 
    SELECT COUNT (DISTINCT Host) AS Hosts_Distintos
    FROM nasa
    """)
host_distintos.show()

+---------------+
|Hosts_Distintos|
+---------------+
|            311|
+---------------+



### Contar número de host únicos cada día

In [65]:
host_distintos_dia = sqlContext.sql(""" 
    SELECT COUNT (DISTINCT Host) AS Hosts_Distintos_Dia
    FROM nasa
    GROUP BY Date
    """)
host_distintos_dia.show()

+-------------------+
|Hosts_Distintos_Dia|
+-------------------+
|                311|
+-------------------+



Sólo nos da el resultado de un día porque tenemos una base de datos simplificada que únicamente contiene una fecha

### Media de peticciones diarias por host

In [64]:
media_peticiones = sqlContext.sql(""" 
    SELECT AVG (Size) AS Media_Peticiones, Host
    FROM nasa
    GROUP BY Host
    """)
media_peticiones.show()

+------------------+--------------------+
|  Media_Peticiones|                Host|
+------------------+--------------------+
|            6309.0|ix-sea6-23.ix.net...|
| 1701.111111111111|grimnet23.idirect...|
|           79536.8|        hella.stm.it|
|            3260.0|       198.161.85.36|
|            2533.0|ix-sd6-29.ix.netc...|
| 6860.066666666667|     info.telenor.no|
|            3949.6|dd08-029.compuser...|
|           27739.0|engei.engei-hs.oy...|
|           10371.0|  server.indo.net.id|
|           19852.0|in2pc2.med.niigat...|
|            547.75|ix-sf10-28.ix.net...|
|          16671.75|ip-pdx4-15.telepo...|
| 13894.90909090909|   in24.inetnebr.com|
|          4216.375|  pwestec.sierra.net|
|           3573.75|roseanne06.slip.y...|
|10749.333333333334|     204.199.188.113|
|            2533.0|        206.24.43.11|
|            3260.0|empedocles.cfar.u...|
| 3448.818181818182|     pm6a3.sover.net|
|  8894.57142857143|www-b5.proxy.aol.com|
+------------------+--------------

### Lista de 40 endpoints distintos que generan un código de respuesta 404

In [67]:
cuarenta_endpoints_respuesta_a0a = sqlContext.sql(""" 
    SELECT Endpoint, Code
    FROM nasa
    WHERE Code = 404
    LIMIT 40
    """)
cuarenta_endpoints_respuesta_a0a.show()

+--------------------+----+
|            Endpoint|Code|
+--------------------+----+
|/shuttle/resource...| 404|
|/pub/winvn/releas...| 404|
|/www/software/win...| 404|
|/history/history.htm| 404|
|/elv/DELTA/uncons...| 404|
|     /sts-71/launch/| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/pub/winvn/releas...| 404|
|/pub/winvn/readme...| 404|
|/pub/winvn/releas...| 404|
+--------------------+----+
only showing top 20 rows



### Top 25 de endpoints que más códigos de respuesta 404 generan

In [102]:
top_endpoints_respuesta_a0a = sqlContext.sql(""" 
    SELECT Endpoint, Code
    FROM nasa
    WHERE Code = 404
    ORDER BY Size DESC
    LIMIT 25
    """)
top_endpoints_respuesta_a0a.show()

+--------------------+----+
|            Endpoint|Code|
+--------------------+----+
|/history/history.htm| 404|
|/elv/DELTA/uncons...| 404|
|     /sts-71/launch/| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/history/apollo/a...| 404|
|/pub/winvn/releas...| 404|
|/pub/winvn/readme...| 404|
|/pub/winvn/releas...| 404|
|/pub/winvn/readme...| 404|
|/pub/winvn/releas...| 404|
|/shuttle/resource...| 404|
+--------------------+----+
only showing top 20 rows



### Top 5 de días que se generaron códigos de respuesta 404

In [ ]:
top_dias_respuesta_a0a = sqlContext.sql(""" 
    SELECT Date, Code
    FROM nasa
    WHERE Code = 404
    ORDER BY Size
    GROUP BY Date
    LIMIT 5
    """)
top_dias_respuesta_a0a.show()